# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

from googletrans import Translator

# MyDataFrame Class

In [ ]:
class MyDataFrame: 
    def __init__(self, df, translate=False, translate_first_level=False, white_space=False, drop_level=True):
        
        self.df = df
        self.translator = Translator()
        self.title = ''
        self.unit = ''
        self.footer = ''
        self.translate = translate
        self.translate_first_level = translate_first_level
        self.white_space= white_space
        self.drop_level = drop_level
        #self.ratio = (((self.df.iloc[:,-2]-self.df.iloc[:,-3])/self.df.iloc[:,-3])*100).round()
        #self.last_column = self.df.iloc[:,-1].replace('..', 0).round()
        
        self.drop_na()
        
        if self.df.index.nlevels > 1:
            self.title_unit_multiindex();
        
        if self.drop_level:
            self.drop_levels()
            
        #if (self.ratio == self.last_column).all():
        #   self.drop_last_column()
        
        
        if self.translate:
            if self.df.index.nlevels == 1:
                self.translate_index()
                
            
        if self.translate:
            if self.df.index.nlevels > 1:
                self.translate_multi_index()
        
        
        if self.white_space:
            self.replace_white_space()
            
            
    def title_unit_multiindex(self):
        self.title = self.df.columns[0][0]
        self.unit = self.df.columns[1][1]
            
    def drop_levels(self):
        """
        This method drops two column levels that contained the infos previously captured (table's title and unit)
        """     
        while self.df.columns.nlevels>1:
            self.df.columns = self.df.columns.droplevel(0)
            
            
    def drop_na(self):
        """
        This method drops all rows and columns that have all values equals to NaN.
        """  
        self.df.dropna(how = 'all', inplace = True)
        self.df.dropna(axis = 'columns', how = 'all', inplace = True)

    
    def translate_index(self):
        """
        This method translates the index of a DataFrame to English.
        """
        self.new_index = []
        for index in self.df.index:
            if index == 'Reinjeção':
                self.new_index.append('Reinjection')
            elif (index == 'Espírito Santo') or (index == 'Espirito_Santo'):
                self.new_index.append('Espirito_Santo')
            elif index == 'Amazonas':
                self.new_index.append('Amazonas')
            elif index == 'Alagoas':
                self.new_index.append('Alagoas')
            elif (index == 'Ceará') or (index == 'Ceara'):
                self.new_index.append('Ceara')
            elif (index == 'Rio Grande do Norte') or (index == 'Rio_Grande_do_Norte'):
                self.new_index.append('Rio_Grande_do_Norte')
            else:
                self.new_index.append(self.translator.translate(index).text)
        self.df.index = self.new_index
        
                        
    def translate_multi_index(self):
        """
        This method translates a MultiIndex DataFrame to English.
        """
        
        if self.translate_first_level == True:
            for i, num in enumerate(self.df.index):
                    for j in range(self.df.index.nlevels):       
                        if j==0:
                            if (self.df.index[i][j] == 'Espírito_Santo') or (self.df.index[i][j] == 'Espirito_Santo'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Espírito_Santo','Espirito_Santo'), level = j)
                            elif self.df.index[i][j] == 'Amazonas':
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Amazonas','Amazonas'), level = j)
                            elif self.df.index[i][j] == 'Alagoas':
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Alagoas','Alagoas'), level = j)
                            elif (self.df.index[i][j] == 'Ceará') or (self.df.index[i][j] == 'Ceara'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Ceará','Ceara'), level = j)
                            elif (self.df.index[i][j] == 'Rio Grande do Norte') or (self.df.index[i][j] == 'Rio_Grande_do_Norte'):
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(' ','_'), level = j)
                            else:
                                self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(self.df.index[i][j], self.translator.translate(self.df.index[i][j]).text), level = j)
        
        for i, num in enumerate(self.df.index):
            for j in range(self.df.index.nlevels):
                if j==0:
                    pass
                if j==1:
                    if self.df.index[i][j] == 'Mar': # checks if one of the words that the translate package can not translate
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Mar','Offshore'), level = j)
                    elif self.df.index[i][j] == 'Terra': # checks if one of the words that the translate package can not translate
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace('Terra','Onshore'), level = j)
                    elif not isinstance(self.df.index[i][j], str):
                        pass
                    else:
                        self.df.index = self.df.index.set_levels(self.df.index.levels[j].str.replace(self.df.index[i][j], self.translator.translate(self.df.index[i][j]).text), level = j)
        
        
    def replace_underscore(self):
        """
        This method replaces all underscore for white space.
        """
        if self.df.index.nlevels > 1: # tells how many level are
            for i, level in enumerate(range(self.df.index.nlevels)): # runs through levels
                #for j, value in enumerate(self.df.index.levels[i]): # runs through the level's value and replace white space for underline
                self.df.index = self.df.index.set_levels(self.df.index.levels[i].str.replace("_", " "), level = i)
        
        elif self.df.index.nlevels == 1:
            self.new_index = []
            for index in self.df.index:
                self.new_index.append(index.replace('_', ' '))
            self.df.index = self.new_index
            
            
    def drop_unnamed_column(self):
        """
        This method drops the last column if its name starts with 'Unnamed'.
        """            
        for i,name in enumerate(self.df.columns):
            if type(name) == str and name.startswith('Unnamed'):
                self.df = self.df.drop(self.df.columns[-1], axis=1)

    def index_sups(self):
        if self.df.index.nlevels > 1:
            for name in self.df.index.levels[0]:

                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('1','¹'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('2','²'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('3','³'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('4','⁴'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('5','⁵'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('6','⁶'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('7','⁷'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('8','⁸'), level = 0)
                self.df.index = self.df.index.set_levels(self.df.index.levels[0].str.replace('9','⁹'), level = 0)

        if self.df.index.nlevels == 1:
            self.df.index = self.df.index.str.replace('1','¹')
            self.df.index = self.df.index.str.replace('2','²')
            self.df.index = self.df.index.str.replace('3','³')
            self.df.index = self.df.index.str.replace('4','⁴')
            self.df.index = self.df.index.str.replace('5','⁵')
            self.df.index = self.df.index.str.replace('6','⁶')
            self.df.index = self.df.index.str.replace('7','⁷')
            self.df.index = self.df.index.str.replace('8','⁸')
            self.df.index = self.df.index.str.replace('9','⁹')



# Load and Wrangling

### Total¹ Reserves of Natural Gas, by Location (Shore and Offshore), according to Federation² Units - 2010-2019

In [ ]:
total_reserves = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_5.xls',  header = [0,2,3], index_col = [0,1]), translate=True,)

In [ ]:
#dropping Unnamed column
total_reserves.drop_unnamed_column()

# Correcting Index Supscrit
total_reserves.index_sups()

# Setting unit, title and footer
total_reserves.unit = '10⁶ m³'
total_reserves.title = 'Total¹ Reserves of Natural Gas, by Location (Shore and Offshore), according to Federation² Units - 2010-2019'
total_reserves.footer ='Source:\nANP/SDP, as per Resolution ANP No. 47/2014.\n\nNotes:\n1. Reserves on 12/31 of the reference years.\n2. See the General Notes item on "Brazilian Oil and Natural Gas Reserves".\n\n¹ Including reserves whose fields Development Plans are under analysis.\n² The reserves are fully appropriated to the state in which each field has its area mostly located.\n³ The Roncador and Frade field reserves are fully appropriated in the State of Rio de Janeiro, for simplification.\n⁴ The Sapinhoá field reserves are fully appropriated in the State of São Paulo for simplification.\n⁵ The reserves in the Caravela field are fully appropriated in the State of Paraná, for simplification.\n⁶ The Tubarão field reserves are fully appropriated in the State of Santa Catarina, for simplification.\n'

### Evolution of Natural Gas Processing Capacity, According to Production Centers - 2010-2019

In [ ]:
processing = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_30.xls',  header = [0,2,3], index_col = [0]))

In [ ]:
# Setting unit, title and footer
processing.unit = '10³ m³/dia'
processing.title = 'Evolution of Natural Gas Processing Capacity, According to Production Centers - 2010-2019'
processing.footer = '¹ Volume in the gaseous state.\n² Includes the UPGNs (Natural Gas Production Units) in Catu and Bahia until 2013. From 2014, only includes Catu.'

### Proved natural gas reserves¹, per location (onshore and offshore), by State² – 2010-2019

In [ ]:
proved_reserves = MyDataFrame(pd.read_excel(r'data_set/table-1_2.xls', header = [0,2,3], index_col = [0,1]))

In [ ]:
# Dropping Unnamed Column
proved_reserves.drop_unnamed_column()

# Correcting Index Supscrit
proved_reserves.index_sups()


proved_reserves.unit = '10⁶ m³'
proved_reserves.title = 'Proved natural gas reserves¹, per location (onshore and offshore), by State² – 2010-2019'
proved_reserves.footer = 'Source: \nANP/SDP, as per Ordinance ANP No. 47/2014.\n\nNotes: \n1. Reserves on December 31 of reference years.1Reserves related to fields whose development plans are still under analysis by ANP included.\n2. Condensates included.\n3. See item in General Notes on "Brazilian Oil and Natural Gas Reserves".\n\n¹ Reserves related to fields whose development plans are still under analysis by ANP included.\n² The reserves are fully appropriated to the State in which each field has its area.\n³ Reserves related to Roncador and Frade fields were totally accounted to the State of Rio de Janeiro by means of simplification.\n⁴ Sapinhoá field reserves are fully appropriated in the State of São Paulo for simplification.\n⁵ Reserves related to Caravela field were totally accounted to the State of Paraná by means of simplification.\n⁶ Reserves related to Tubarão field are totally accounted to the State of Santa Catarina by means of simplification.'

### Natural Gas Domestic Sales by Brazilian Region and State – 2010-2019

In [ ]:
domestic_sale = MyDataFrame(pd.read_excel(r'data_set/table-1_25.xls', header = [0,2,3], index_col = [0]))

In [ ]:
# Dropping Unnamed Column
domestic_sale.drop_unnamed_column()

# Correcting Index Supscrit
domestic_sale.index_sups()

# Setting unit, title and footer
domestic_sale.unit = '10⁶ m³'
domestic_sale.title = 'Natural Gas Domestic Sales by Brazilian Region and State – 2010-2019'
domestic_sale.footer = 'Sources: \nPetrobras and ANP.\n\nNote:\nRelated only the States where there were sales of natural gas in the specified period.\n\n¹Include sales to thermal generation ²Sales for Nitrogen Fertilizer plants (Fafen) and sales for thermal generation.'

### Brazilian natural gas balance – 2010-2019

In [ ]:
balance = MyDataFrame(pd.read_excel(r'data_set/table-1_26.xls', header = [0,2,3], index_col = [0]))

In [ ]:
# Dropping Unnamed Column
balance.drop_unnamed_column()

# Correcting Index Supscrit
balance.index_sups()

# Setting unit, title and footer
balance.unit = '10⁶ m³'
balance.title = 'Brazilian natural gas balance – 2010-2019'
balance.footer = 'Sources: \nANP/SIM, as per Ordinance ANP No. 43/98, for imports data; ANP/SDP, as per Decree No. 2.705/98, for  production, reinjection, gas flaring and losses data; Petrobras, for own consumption, NGL and sales data.\n\n¹ Refers to Petrobras own consumption in production areas, refineries, NGPP (Natural Gas Power Plant), transportation and storage. \n² Volume of gas absorbed in NGPPs. \n³ Sales to distributors, nitrofertilizers plants (Fafen) and electricity generation.'

### Brazilian Natural Gas Demand – 2010-2019

In [191]:
balance.df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Import,12647.163059,10481.230275,13142.787000,16513.465000,17398.010000,19111.557000,13320.723000,10642.550000,10842.398805,9855.431833
Export,0.000000,50.220631,312.300000,37.381800,90.466000,1.870000,517.481000,134.520000,80.136042,0.000000
Production,22938.446513,24073.730577,25832.244517,28174.212905,31894.874000,35126.447000,37890.450000,40117.402000,40857.207060,44724.231944
Reinjection,4369.053258,4037.731513,3542.732598,3883.004317,5739.690000,8866.713000,11069.494474,10076.841000,12811.951061,15776.669711
Gas flaring,2417.758896,1756.226339,1444.516950,1302.885282,1619.208000,1397.675000,1484.082000,1377.136000,1358.206104,1589.738474
Own consumption¹,6744.762530,7802.974057,8850.031417,9077.946961,9335.072936,10851.056777,9359.798000,9630.178814,9352.726619,9393.638796
NGL²,1335.038659,1286.725075,1281.447149,1336.773000,1505.328000,1380.795000,1540.860000,1851.088000,1896.362000,1960.897610
Sales³,20457.563469,19307.265264,23284.159076,28783.642605,30768.358064,31502.467702,27224.483515,27490.860364,26050.725307,25854.502470
Adjustments and losses,261.432759,313.817972,259.844328,266.043941,234.761000,237.426521,14.974011,199.327822,149.498732,4.216716
Adjustments and losses,261.432760,313.817972,259.844327,266.043940,234.761000,237.426521,14.974011,199.327822,149.498732,4.216716


In [197]:
# Demand DataFrame
demand = MyDataFrame(balance.df.iloc[[1,3,4,5,6,7]].T)

# Setting unit, title and footer
demand.title = 'Brazilian Natural Gas Demand by Consumer – 2010-2019'
demand.unit = '10⁶ m³'
demand.footer = 'Sources: \nANP/SIM, as per Ordinance ANP No. 43/98, for imports data; ANP/SDP, as per Decree No. 2.705/98, for  production, reinjection, gas flaring and losses data; Petrobras, for own consumption, NGL and sales data.\n\n¹ Refers to Petrobras own consumption in production areas, refineries, NGPP (Natural Gas Power Plant), transportation and storage. \n² Volume of gas absorbed in NGPPs. \n³ Sales to distributors, nitrofertilizers plants (Fafen) and electricity generation.'

In [201]:
# Demand Total Series
demand_total = demand.df.sum(axis=1)

# Setting unit, title and footer
demand_total_title = 'Brazilian Natural Gas Total Demand – 2010-2019'
demand_total_unit = '10⁶ m³'

### Natural gas production, per location (onshore and offshore, pre-salt and post-salt), by State – 2010-2019 

In [ ]:
production = MyDataFrame(pd.read_excel(r'data_set/table-1_5.xls', header = [0,2,3], index_col = [0,1]))

In [ ]:
# Dropping Unnamed Column
production.drop_unnamed_column()

# Correcting Index Supscrit
production.index_sups()

# Setting unit, title and footer
production.unit='10⁶ m³'
production.title = 'Natural gas production, per location (onshore and offshore, pre-salt and post-salt), by State – 2010-2019'
production.footer = 'Source: \nANP/SDP, as per Decree No. 2.705/1998.\n\nNote: \nTotal production includes reinjection, gas flaring, losses and own consumption.'

### Production of associated and non-associated natural gas, by State – 2010-2019

In [ ]:
gas_type = MyDataFrame(pd.read_excel(r'data_set/table-1_6.xls', header = [0,2,3], index_col = [0,1]))

In [ ]:
# Dropping Unnamed Column
gas_type.drop_unnamed_column()

# Correcting Index Supscrit


# Setting unit, title and footer
gas_type.unit='10⁶ m³'
gas_type.title = 'Production of associated and non-associated natural gas, by State – 2010-2019'
gas_type.footer = 'Source: \nANP/SDP, as per Decree No. 2.705/1998.\n\nNote: \nTotal production volume includes reinjection, flared gas, losses, own consumption.'

### Average Reference Prices for Natural Gas, According to Federation Units - 2010-2019

In [ ]:
avg_price = MyDataFrame(pd.read_excel(r'data_set/anuario-2020-tabela-2_24.xls',  header = [0,2,3,4], index_col = [0]))

# Slicing 'US$/mil m³' units only
avg_price.df = avg_price.df.iloc[:,10:20]

In [ ]:
# Setting unit, title and footer
avg_price.unit='US$/mil m³'
avg_price.title = 'Average Reference Prices for Natural Gas, According to Federation Units - 2010-2019'
avg_price.footer = 'Source: \nANP / SPG, according to Law No. 9,478 / 1997, Decree No. 2,705 / 1998 and ANP Ordinance No. 206/2000. \n\nNotes: \n1. Prices in current values. \n2. Only the Federation units that presented natural gas production in the indicated period are listed. \n3. The above prices do not serve as a basis for calculating government stakes, since they are weighted averages only for production volumes per field and do not consider royalty and special participation rates per producing field.'

### Electric Energy by Source

In [184]:
energy_source = pd.read_csv(r'data_set/GeracaoFonte.csv')
energy_source.columns = ['ID', 'Energy Type', 'GWh', 'Month', 'Year', 'dthProx']
energy_source = MyDataFrame(pd.pivot_table(energy_source, values='GWh', index=['Energy Type'], columns=['Year', 'Month']), drop_level=False)

In [186]:
# Setting unit, title and footer
energy_source.unit = 'GWh'
energy_source.title = 'Electric Energy by Source'

### Brazilian Indicators

In [189]:
bra_ind = pd.read_csv(r'data_set/API_BRA_DS2_en_csv_v2_1221399.csv', skiprows=[0,1,2,3])